# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_GEMINI = 'gemini-2.5-pro'

In [5]:
# set up environment
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

if openai_api_key and openai_api_key.startswith("sk-proj-") and len(openai_api_key)>10:
    print("Open AI Api key looks good. Lets keep going\nChecking Gemini....")
    if gemini_api_key and gemini_api_key.startswith("AIzaSyC") and len(gemini_api_key)>10:
        print("Everything looks good so far. Lets keep going")

Open AI Api key looks good. Lets keep going
Checking Gemini....
Everything looks good so far. Lets keep going


In [15]:
system_prompt = """
You're an experienced code reviewer tasked with helping newbie developers understand what their code does. 
Your job is to explain each code step-by-step while keeping explanations as simple as possible so the newbies can understand.
Display your answer in markdown
"""

In [ ]:
user_prompt=""""""

In [11]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

def get_question(question):
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt+question}]
    return messages 


In [12]:
get_question(question)

[{'role': 'system',
  'content': "\nYou're an experienced code reviewer tasked with helping newbie developers understand what their code does. \nYour job is to explain each code step-by-step while keeping explanations as simple as possible so the newbies can understand.\n"},
 {'role': 'user',
  'content': '\n\nPlease explain what this code does and why:\nyield from {book.get("author") for book in books if book.get("author")}\n'}]

In [ ]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_api_key)




In [23]:
def get_answer(question):
    gpt_response = openai.chat.completions.create(model=MODEL_GPT, messages=get_question(question))
    chatgpt_answer = gpt_response.choices[0].message.content

    gemini_response = gemini.chat.completions.create(model=MODEL_GEMINI, messages=get_question(question))
    gemini_answer = gemini_response.choices[0].message.content

    display(Markdown("# *ChatGPT's thoughts*"))
    display(Markdown(chatgpt_answer))
    display(Markdown("# *Gemini's thoughts*"))
    display(Markdown(gemini_answer))

In [24]:
# Get Llama 3.2 to answer
get_answer(question)

# *ChatGPT's thoughts*

Sure! Let's break down the code step-by-step.

### Code Breakdown

```python
yield from {book.get("author") for book in books if book.get("author")}
```

#### 1. **Understanding `books`**
- `books` is expected to be a collection (like a list) that contains multiple items, where each item is a dictionary representing a book. For example, a book might look like this:
  ```python
  {
      "title": "The Great Gatsby",
      "author": "F. Scott Fitzgerald"
  }
  ```

#### 2. **List Comprehension**
- The part inside the curly braces `{...}` is a **set comprehension**. It's similar to a list comprehension but creates a set instead of a list.
- The code that is run inside the set comprehension is:
  ```python
  book.get("author") for book in books if book.get("author")
  ```
  
- **Breaking it down further:**
  - `for book in books`: This loops through each `book` in the `books` collection.
  - `book.get("author")`: This tries to fetch the value associated with the key `"author"` from the `book` dictionary. If the author exists, it returns the author's name; if not, it returns `None`.
  - `if book.get("author")`: This checks if the author exists (i.e., it's not `None`). If it exists, the author is included; if not, it is skipped.

#### 3. **Set Creation**
- The entire set comprehension creates a **set** of unique authors for the books where authors exist. A set is useful here because it automatically handles duplicates (if multiple books have the same author, that author will only appear once in the set).

#### 4. **Using `yield from`**
- The `yield from` statement is used to yield all values from an iterable (in this case, the set we created).
- This means that instead of returning one item at a time, it allows a generator to return all the unique authors one-by-one.

### Summary
So, in summary, this line of code does the following:
1. It goes through a collection of `books`, checks each book for an existing author.
2. It gathers all unique authors into a set.
3. It uses `yield from` to yield each of these authors one by one, allowing them to be processed individually by the calling code.

### Why Use It?
- This code is useful for extracting and processing all unique authors from a list of books efficiently, particularly when you want to ensure that the final list does not contain any duplicate authors.

# *Gemini's thoughts*

Of course! Let's break down that line of Python code. It's a very concise and efficient piece of code that does a few things at once.

### High-Level Summary

In simple terms, this code finds all the **unique** author names from a list of `books` and passes them out one by one. It's designed to be safe (it won't crash if a book has no author) and memory-efficient.

### Let's Break It Down Step-by-Step

To understand this line, we need to read it from the inside out.

Let's imagine you have a list of book dictionaries like this:

```python
books = [
    {"title": "The Hobbit", "author": "J.R.R. Tolkien"},
    {"title": "Harry Potter", "author": "J.K. Rowling"},
    {"title": "Fantastic Beasts", "author": "J.K. Rowling"}, # A duplicate author
    {"title": "A Book with No Author"}, # A missing author key
    {"title": "Another Book", "author": None} # An author key with no value
]
```

The code we are looking at is: `yield from {book.get("author") for book in books if book.get("author")}`

---

#### Step 1: The Loop `for book in books`

This is the simplest part. It just means "Go through the `books` list, one dictionary at a time."

-   **First loop:** `book` is `{"title": "The Hobbit", "author": "J.R.R. Tolkien"}`
-   **Second loop:** `book` is `{"title": "Harry Potter", "author": "J.K. Rowling"}`
-   ...and so on.

---

#### Step 2: The Condition `if book.get("author")`

For each `book` in the list, this `if` statement checks if the author exists and has a meaningful value.

*   `book.get("author")` is a safe way to get a value from a dictionary.
    *   If the key `"author"` exists, it returns the value (e.g., `"J.R.R. Tolkien"`).
    *   If the key `"author"` **does not exist**, it returns `None` instead of crashing your program. This is why we use `.get()` instead of `book["author"]`.

*   The `if` statement then checks if the result is "truthy". In Python, `None`, `False`, the number `0`, and empty things (like an empty string `""`) are considered "falsy". Everything else is "truthy".
    *   `"J.R.R. Tolkien"` is a non-empty string, so it's **truthy**. The `if` passes.
    *   `None` is **falsy**. The `if` fails.

Let's see how this works on our `books` list:
1.  `book.get("author")` -> `"J.R.R. Tolkien"` (Truthy) -> **Pass**
2.  `book.get("author")` -> `"J.K. Rowling"` (Truthy) -> **Pass**
3.  `book.get("author")` -> `"J.K. Rowling"` (Truthy) -> **Pass**
4.  `book.get("author")` -> `None` (Falsy) -> **Fail** (This book is skipped)
5.  `book.get("author")` -> `None` (Falsy) -> **Fail** (This book is also skipped)

So, this `if` condition is a filter to make sure we only process books that actually have an author's name.

---

#### Step 3: The Expression `{book.get("author") ... }`

This whole part is a **Set Comprehension**. You can spot it by the curly braces `{}`.

*   A **set** is a collection of items where every item must be **unique**. Duplicates are automatically removed.
*   A **comprehension** is a short-hand way to create a collection (like a list or a set) from another collection.

So, for every book that passes the `if` filter, we take the result of `book.get("author")` and add it to our set.

Let's trace it:
1.  First book's author: `"J.R.R. Tolkien"`. The set is now: `{"J.R.R. Tolkien"}`.
2.  Second book's author: `"J.K. Rowling"`. The set is now: `{"J.R.R. Tolkien", "J.K. Rowling"}`.
3.  Third book's author: `"J.K. Rowling"`. The set tries to add it, but since `"J.K. Rowling"` is already there, nothing changes. The set remains: `{"J.R.R. Tolkien", "J.K. Rowling"}`.

After the loop is finished, the set comprehension creates this final set: `{"J.R.R. Tolkien", "J.K. Rowling"}`. Notice how the duplicate author was automatically handled!

---

#### Step 4: The Keyword `yield from`

This is the most advanced part. It tells us this line of code is inside a special kind of function called a **generator**.

*   A normal function uses `return` to send back a single, final result (like our entire set).
*   A generator function uses `yield` to send back values one at a time, pausing in between. This is very memory-efficient for large lists because it doesn't create a giant result list in memory.

What does `yield from` do?
It's a shortcut. It says, "Take the collection that follows (our set), loop through it, and `yield` each item from it one by one."

So, `yield from {"J.R.R. Tolkien", "J.K. Rowling"}` is the same as writing:

```python
# Long-hand version
the_set_of_authors = {"J.R.R. Tolkien", "J.K. Rowling"}
for author in the_set_of_authors:
    yield author
```

The generator will first `yield` `"J.R.R. Tolkien"` (or `"J.K. Rowling"`, as sets have no order), then pause. When asked for the next item, it will resume and `yield` the other name.

### Putting It All Together (The "Why")

So, **why use this specific line of code?**

1.  **It's Safe:** `book.get("author")` prevents the code from crashing if a book dictionary is missing the `"author"` key.
2.  **It's Clean:** The `if book.get("author")` check filters out any books without a valid author name, preventing `None` from being in your final result.
3.  **It's Unique:** Using a set comprehension `{...}` is a simple and highly readable way to get a list of unique authors. No extra code is needed to handle duplicates.
4.  **It's Memory-Efficient:** The `yield from` makes it part of a generator. This means it processes the books and produces the author names on-demand, one by one, instead of building a potentially huge list of all authors in memory first. This is great for working with very large datasets.